In [ ]:
# Setup: install Qiskit (runs automatically in Colab, no-op in Binder)
!pip install -q qiskit qiskit-aer qiskit-ibm-runtime pylatexenc

# CHSH-Ungleichung

*Nötzungsschätzung: Zwei Minute op enem Heron r2-Prozessor (HENWIES: Dat es bloß en Schätzung. Ding Laufzick kann variiere.)*
## Hengerjrund
En däm Tutorial mäht mer e Experiment op enem Quantecomputer, öm de Verletzung vun der CHSH-Ungleichung met dem Estimator-Primitiv ze zeije.

De CHSH-Ungleichung, jenannt noh de Autore Clauser, horne, Shimony un Holt, weed jenötzt, öm Bells Theorem (1969) experimentell ze bewiese. Dat Theorem sät us, dat lokale Theorie met verborgene Variable nit all Konsequenze vun der Verschränkung en der Quantemechanik erkläre künne. De Verletzung vun der CHSH-Ungleichung weed jenötzt, öm ze zeije, dat de Quantemechanik met lokale Theorie met verborgene Variable nit vereinbar es. Dat es e wichtich Experiment för et Verstendnis vun de Jrundlage vun der Quantemechanik.

Dä Nobelpreis för Physik 2022 wood an Alain Aspect, John Clauser un Anton Zeilinger verjovve, unger anderem för hör Pionierarbeit en der Quanteinformationswessenschaft un besonders för hör Experimente met verschränkte Photone, die de Verletzung vun de Bellsche Ungleichunge jezeeht han.
## Aanforderunge
Vör däm Aanfang vun däm Tutorial, stell sescher, dat do Foljendes installeet häs:

* Qiskit SDK v1.0 oder neuher, met [visualization](https://docs.quantum.ibm.com/api/qiskit/visualization)-Ungerstötzung
* Qiskit Runtime (`pip install qiskit-ibm-runtime`) v0.22 oder neuher
## Opbou

In [1]:
# General
import numpy as np

# Qiskit imports
from qiskit import QuantumCircuit
from qiskit.circuit import Parameter
from qiskit.quantum_info import SparsePauliOp
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

# Qiskit Runtime imports
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_runtime import EstimatorV2 as Estimator

# Plotting routines
import matplotlib.pyplot as plt
import matplotlib.ticker as tck

## Schrett 1: Klassische Enjaave op e Quanteproblem afbelde
För dat Experiment maache mer e verschränk Paar, wo mer jedes Qubit op zwei verschiedene Baase messe. Mer kennzeechne de Baase för et eezte Qubit met $A$ un $a$ un de Baase för et zweite Qubit met $B$ un $b$. Dat erlöv et uns, de CHSH-Jrüß $S_1$ ze berechne:

$$
S_1 = A(B-b) + a(B+b).
$$

Jede Observable es entweder $+1$ oder $-1$. Klor es, dat eine vun de Terme $B\pm b$ jlich $0$ sin muss un der ander $\pm 2$ sin muss. Doröm es $S_1 = \pm 2$. Dä Durchschnettswäät vun $S_1$ muss de Ungleichung erfülle:

$$
|\langle S_1 \rangle|\leq 2.
$$

Wenn mer $S_1$ en Bezoch op $A$, $a$, $B$ un $b$ uswiggelt, krieje mer:

$$
|\langle S_1 \rangle| = |\langle AB \rangle - \langle Ab \rangle + \langle aB \rangle + \langle ab \rangle| \leq 2
$$

Do kanns noch en wigger CHSH-Jrüß $S_2$ deffiniere:

$$
S_2 = A(B+b) - a(B-b),
$$

Dat föhrt zo ener wigger Ungleichung:

$$
|\langle S_2 \rangle| = |\langle AB \rangle + \langle Ab \rangle - \langle aB \rangle + \langle ab \rangle| \leq 2
$$

Wenn de Quantemechanik durch lokale Theorie met verborgene Variable beschrevve wääde kann, müsse de vörherije Ungleichunge wohr sin. Ävver wie en däm Tutorial jezeeht weed, künne dise Ungleichunge op enem Quantecomputer verletzt wääde. Doröm es de Quantemechanik nit met lokale Theorie met verborgene Variable vereinbar.
Wann do mieh Theorie liere wells, lohr der [Entanglement in Action](/learning/courses/basics-of-quantum-information/entanglement-in-action/chsh-game) met em John Watrous aan.
Mer maache e verschränk Paar zwesche zwei Qubits en enem Quantecomputer, endäm mer dä Bell-Zostand $|\Phi^+\rangle = \frac{|00\rangle + |11\rangle}{\sqrt{2}}$ erzööje. Met däm Estimator-Primitiv kanns de direkt de nüdije Erwartungswääte ($\langle AB \rangle, \langle Ab \rangle, \langle aB \rangle$ un $\langle ab \rangle$) krijje, öm de Erwartungswääte vun de beede CHSH-Jrüße $\langle S_1\rangle$ un $\langle S_2\rangle$ ze berechne. Vör der Enföhrung vum Estimator-Primitiv hätts de de Erwartungswääte us de Messerjebness konstruiere müsse.

Mer messe et zweite Qubit en de $Z$- un $X$-Baase. Et eezte Qubit weed och en orthogonale Baase jemesse, ävver met enem Wenkel bezöhlich vum zweite Qubit, dä mer zwesche $0$ un $2\pi$ variiere wääde. Wie de siehs, mäht et Estimator-Primitiv et Usföhre vun parametrisierte Schaltkreise sehr einfach. Anstatt en Reihe vun CHSH-Schaltkreise ze maache, moß mer bloß *eine* CHSH-Schaltkreis met enem Parameter maache, dä dä Messwinkel aanjitt, un en Reihe vun Phasewääte för dä Parameter.

Schleßlich wääde mer de Erjebness analysiere un jeje dä Messwinkel opträje. Do weeß sinn, dat för en bestemmte Bereich vun Messwenkele de Erwartungswääte vun de CHSH-Jrüße $|\langle S_1\rangle| > 2$ oder $|\langle S_2\rangle| > 2$ sin, wat de Verletzung vun der CHSH-Ungleichung zeecht.

In [2]:
# To run on hardware, select the backend with the fewest number of jobs in the queue
service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, simulator=False, min_num_qubits=127
)
backend.name

'ibm_kingston'

### Create a parameterized CHSH circuit

First, we write the circuit with the parameter $\theta$, which we call `theta`. The [`Estimator` primitive](https://docs.quantum-computing.ibm.com/api/qiskit-ibm-runtime/qiskit_ibm_runtime.EstimatorV2) can enormously simplify circuit building and output analysis by directly providing expectation values of observables. Many problems of interest, especially for near-term applications on noisy systems, can be formulated in terms of expectation values. `Estimator` (V2) primitive can automatically change measurement basis based on the supplied observable.

In [3]:
theta = Parameter("$\\theta$")

chsh_circuit = QuantumCircuit(2)
chsh_circuit.h(0)
chsh_circuit.cx(0, 1)
chsh_circuit.ry(theta, 0)
chsh_circuit.draw(output="mpl", idle_wires=False, style="iqp")

<Image src="../docs/images/tutorials/chsh-inequality/extracted-outputs/6c77e40a-0.avif" alt="Output of the previous code cell" />

### Ene parametrisierte CHSH-Schaltkreis maache
Eets schrieve mer dä Schaltkreis met däm Parameter $\theta$, dä mer `theta` nenne. Dat [`Estimator`-Primitiv](https://docs.quantum-computing.ibm.com/api/qiskit-ibm-runtime/qiskit_ibm_runtime.EstimatorV2) kann dä Schaltkreisopbou un de Usjaabeanalyse ärm vereinfache, endäm et direkt Erwartungswääte vun Observable leffert. Fill interessante Probleme, besonders för korzfristije Aanwendunge op verrauschte Systeme, künne en Form vun Erwartungswääte formuleet wääde. Dat `Estimator` (V2)-Primitiv kann automatisch de Messbasis basierend op der bereetjestellte Observable ändere.

In [4]:
number_of_phases = 21
phases = np.linspace(0, 2 * np.pi, number_of_phases)
# Phases need to be expressed as list of lists in order to work
individual_phases = [[ph] for ph in phases]

![Output of the previous code cell](../docs/images/tutorials/chsh-inequality/extracted-outputs/6c77e40a-0.avif)

### En Liss vun Phasewääte maache, die späder zojewiese wääde
Nohdem do dä parametrisierte CHSH-Schaltkreis jemaat häs, mähs de en Liss vun Phasewääte, die däm Schaltkreis em nächste Schrett zojewiese wääde. Do kanns dä folgende Code nötze, öm en Liss vun 21 Phasewääte em Bereich vun $0$ bes $2 \pi$ met jlichem Afstand ze maache, also $0$, $0.1 \pi$, $0.2 \pi$, ..., $1.9 \pi$, $2 \pi$.

In [5]:
# <CHSH1> = <AB> - <Ab> + <aB> + <ab> -> <ZZ> - <ZX> + <XZ> + <XX>
observable1 = SparsePauliOp.from_list(
    [("ZZ", 1), ("ZX", -1), ("XZ", 1), ("XX", 1)]
)

# <CHSH2> = <AB> + <Ab> - <aB> + <ab> -> <ZZ> + <ZX> - <XZ> + <XX>
observable2 = SparsePauliOp.from_list(
    [("ZZ", 1), ("ZX", 1), ("XZ", -1), ("XX", 1)]
)

### Observable
Jetz bruche mer Observable, us dänne mer de Erwartungswääte berechne künne. En unserm Fall betraachte mer orthogonale Baase för jedes Qubit, wobei de parametrisierte $Y$-Rotation för et eezte Qubit de Messbasis fascht kontinuierlich bezöhlich vun der Basis vum zweite Qubit varieet. Mer wähle doröm de Observable $ZZ$, $ZX$, $XZ$ un $XX$.

In [6]:
target = backend.target
pm = generate_preset_pass_manager(target=target, optimization_level=3)

chsh_isa_circuit = pm.run(chsh_circuit)
chsh_isa_circuit.draw(output="mpl", idle_wires=False, style="iqp")

<Image src="../docs/images/tutorials/chsh-inequality/extracted-outputs/9a5561eb-0.avif" alt="Output of the previous code cell" />

## Schrett 2: Problem för de Usföhrung op Quantehardware optimiere

Öm de Jesamtusföhrungszick vum Job ze reduziere, akzeptiere V2-Primitive bloß Schaltkreise un Observable, die de vum Zielsystem ungerstötzte Aanwiesunge un der Konnektivität entsprääche (bezeeechnet als Instruction Set Architecture (ISA)-Schaltkreise un -Observable).

### ISA-Schaltkreis

In [7]:
isa_observable1 = observable1.apply_layout(layout=chsh_isa_circuit.layout)
isa_observable2 = observable2.apply_layout(layout=chsh_isa_circuit.layout)

![Output of the previous code cell](../docs/images/tutorials/chsh-inequality/extracted-outputs/9a5561eb-0.avif)

### ISA-Observable

Ävveso müsse mer de Observable transformiere, öm se backend-kompatibel ze maache, bevör mer Jobs met [`Runtime Estimator V2`](https://docs.quantum.ibm.com/api/qiskit-ibm-runtime/estimator-v2#run) usföhre. Mer künne de Transformation met der `apply_layout`-Methode vum `SparsePauliOp`-Objekt durchföhre.

In [8]:
# To run on a local simulator:
# Use the StatevectorEstimator from qiskit.primitives instead.

estimator = Estimator(mode=backend)

pub = (
    chsh_isa_circuit,  # ISA circuit
    [[isa_observable1], [isa_observable2]],  # ISA Observables
    individual_phases,  # Parameter values
)

job_result = estimator.run(pubs=[pub]).result()

## Schrett 3: Usföhre met Qiskit-Primitive
Öm dat jesampte Experiment en einem einzije Oproof vum [`Estimator`](https://docs.quantum-computing.ibm.com/api/qiskit-ibm-runtime/qiskit_ibm_runtime.EstimatorV2) uszföhre.
Mer künne e [Qiskit Runtime `Estimator`](https://docs.quantum.ibm.com/api/qiskit-ibm-runtime/estimator-v2)-Primitiv maache, öm uns Erwartungswääte ze berechne. De `EstimatorV2.run()`-Methode nemmp e Iterable vun `primitive unified blocs (PUBs)`. Jedes PUB es e Iterable em Format `(circuit, observables, parameter_values: Optional, precision: Optional)`.

In [9]:
chsh1_est = job_result[0].data.evs[0]
chsh2_est = job_result[0].data.evs[1]

In [10]:
fig, ax = plt.subplots(figsize=(10, 6))

# results from hardware
ax.plot(phases / np.pi, chsh1_est, "o-", label="CHSH1", zorder=3)
ax.plot(phases / np.pi, chsh2_est, "o-", label="CHSH2", zorder=3)

# classical bound +-2
ax.axhline(y=2, color="0.9", linestyle="--")
ax.axhline(y=-2, color="0.9", linestyle="--")

# quantum bound, +-2√2
ax.axhline(y=np.sqrt(2) * 2, color="0.9", linestyle="-.")
ax.axhline(y=-np.sqrt(2) * 2, color="0.9", linestyle="-.")
ax.fill_between(phases / np.pi, 2, 2 * np.sqrt(2), color="0.6", alpha=0.7)
ax.fill_between(phases / np.pi, -2, -2 * np.sqrt(2), color="0.6", alpha=0.7)

# set x tick labels to the unit of pi
ax.xaxis.set_major_formatter(tck.FormatStrFormatter("%g $\\pi$"))
ax.xaxis.set_major_locator(tck.MultipleLocator(base=0.5))

# set labels, and legend
plt.xlabel("Theta")
plt.ylabel("CHSH witness")
plt.legend()
plt.show()

<Image src="../docs/images/tutorials/chsh-inequality/extracted-outputs/f6267448-0.avif" alt="Output of the previous code cell" />

In the figure, the lines and gray areas delimit the bounds; the outer-most (dash-dotted) lines delimit the quantum-bounds ($\pm 2$), whereas the inner (dashed) lines delimit the classical bounds ($\pm 2\sqrt{2}$). You can see that there are regions where the CHSH witness quantities exceeds the classical bounds. Congratulations! You have successfully demonstrated the violation of CHSH inequality in a real quantum system!

## Tutorial survey

Please take this short survey to provide feedback on this tutorial. Your insights will help us improve our content offerings and user experience.

[Link to survey](https://your.feedback.ibm.com/jfe/form/SV_3xxAgm1SF1wGp9k)